## Retrieve Data From Wikipedia

This block generates our corpus. That is -
1. download text from Wikipedia pages
2. creates clean text (with not punctuation) = X
3. learn the correct labeling of each word = Y , where $y \in Y$ is $y \in $ {'None','Comma','dot'}

After running this block the corpus = X will be in the file 'pos_to_tag.txt' in the sampe folder of this JN

In [1]:
import os 
import sys
sys.path.append('..\GenerationPipe')
import NLP_HEBPUNCT_GP_generator as gen
import NLP_HEBPUNCT_GP_pre_processing as preproc
import subprocess
from subprocess import check_output

In [2]:


targetUrlList = ["https://he.wikipedia.org/wiki/%D7%A7%D7%A4%D7%99%D7%98%D7%9C%D7%99%D7%96%D7%9D",
                 "https://he.wikipedia.org/wiki/%D7%A7%D7%A8%D7%9C_%D7%9E%D7%A8%D7%A7%D7%A1",
                 "https://he.wikipedia.org/wiki/%D7%90%D7%99%D7%9C%D7%9F_%D7%A8%D7%9E%D7%95%D7%9F",
                 "https://he.wikipedia.org/wiki/%D7%90%D7%9C%D7%91%D7%A8%D7%98_%D7%90%D7%99%D7%99%D7%A0%D7%A9%D7%98%D7%99%D7%99%D7%9F",
                 "https://he.wikipedia.org/wiki/%D7%90%D7%94%D7%91%D7%94",
                 "https://he.wikipedia.org/wiki/%D7%9E%D7%A6%D7%99%D7%90%D7%95%D7%AA",
                 "https://he.wikipedia.org/wiki/%D7%A4%D7%99%D7%9C%D7%95%D7%A1%D7%95%D7%A4%D7%99%D7%94_%D7%A9%D7%9C_%D7%94%D7%9E%D7%93%D7%A2",
                 "https://he.wikipedia.org/wiki/%D7%9B%D7%91%D7%99%D7%93%D7%94",
                 "https://he.wikipedia.org/wiki/%D7%94%D7%9E%D7%90%D7%94_%D7%94-20",
                 "https://he.wikipedia.org/wiki/%D7%9E%D7%96%D7%A8%D7%97_%D7%90%D7%99%D7%A8%D7%95%D7%A4%D7%94",
                 "https://he.wikipedia.org/wiki/%D7%97%D7%95%D7%A7_%D7%94%D7%94%D7%A1%D7%93%D7%A8%D7%99%D7%9D",
                 "https://he.wikipedia.org/wiki/%D7%94%D7%94%D7%A1%D7%AA%D7%93%D7%A8%D7%95%D7%AA_%D7%94%D7%9B%D7%9C%D7%9C%D7%99%D7%AA_%D7%A9%D7%9C_%D7%94%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%9D_%D7%91%D7%90%D7%A8%D7%A5_%D7%99%D7%A9%D7%A8%D7%90%D7%9C",
                 "https://he.wikipedia.org/wiki/%D7%90%D7%93%D7%A8%D7%99%D7%9B%D7%9C%D7%95%D7%AA_%D7%92%D7%95%D7%AA%D7%99%D7%AA",
                 "https://he.wikipedia.org/wiki/%D7%9B%D7%A0%D7%A1%D7%99%D7%99%D7%AA_%D7%94%D7%91%D7%A9%D7%95%D7%A8%D7%94",
                 "https://he.wikipedia.org/wiki/%D7%94%D7%9E%D7%97%D7%9C%D7%95%D7%A7%D7%AA_%D7%A2%D7%9C_%D7%94%D7%98%D7%A7%D7%A1%D7%99%D7%9D_%D7%94%D7%A1%D7%99%D7%A0%D7%99%D7%99%D7%9D"]




def word2features(word,i,pos_dict,g_index):
    
    features = {
        'bias': 1.0,
        'word[:1]': word[:1],
        'word[:2]': word[:2],
        'word[-2:]': word[-2:],
        'word.isdigit()': word.isdigit(),
        'index': i,
        'pos': pos_dict[g_index].split("	")[1].rstrip()
    }
    
    return features

def words2features(words_list,pos_dict,g_index):
    vectors = []
    for i in range(len(words_list)):
        vectors.append(word2features(words_list[i],i,pos_dict,g_index))
        g_index+=1
    return vectors

def texts_to_files(lst,fname):
    file = open("..\corpus\\" + fname + "_words_to_pos.txt","w",encoding="utf-8") 
    for x in lst:
        s = x[:x.index("<XYDEL>")]
        file.write(s + " ") 
    file.close() 

def get_pos_dict(fname):
    file = open("..\corpus_out\\" + fname + "_out.txt", "r", encoding="utf-8") 
    s = file.read().split(" ") 
    pos_list = []
    i=0
    while i < (len(s)):
        pos_list.append(s[i])
        i=i+1
    return pos_list
        
    
    
def generate_dataset(url_list,outname):
    gen.generate(url_list,outname)
    out = gen.getStringFromDataFile(outname)
    out = out.split("<DATADEL>")
    texts_to_files(out,outname)
    data = []
    for x in out:
        data.append(x.split("<XYDEL>"))
    counter = 0
    pos_dict = get_pos_dict(outname)
    dataset=[]
    for d in data:
        temp = []
        temp.append(d[0].split(" "))
        temp.append(d[1].split(" "))
        dataset.append(temp)
    g_index = 0
    for i in range(len(dataset)):
        dataset[i][0] = words2features(dataset[i][0],pos_dict,g_index)
        g_index += len(dataset[i][0])
    return dataset

data = generate_dataset(gen.readyUrlList,"train_data")
x_train = [x[0] for x in data]
y_train = [x[1] for x in data]

print("-----------------------------------------------------")
print(x_train[0])
print("-----------------------------------------------------")
print(y_train[0])

adding to data set URL: https://he.wikipedia.org/wiki/%D7%A7%D7%A4%D7%99%D7%98%D7%9C%D7%99%D7%96%D7%9D
adding to data set URL: https://he.wikipedia.org/wiki/%D7%A7%D7%A8%D7%9C_%D7%9E%D7%A8%D7%A7%D7%A1
adding to data set URL: https://he.wikipedia.org/wiki/%D7%90%D7%99%D7%9C%D7%9F_%D7%A8%D7%9E%D7%95%D7%9F
adding to data set URL: https://he.wikipedia.org/wiki/%D7%90%D7%9C%D7%91%D7%A8%D7%98_%D7%90%D7%99%D7%99%D7%A0%D7%A9%D7%98%D7%99%D7%99%D7%9F
adding to data set URL: https://he.wikipedia.org/wiki/%D7%A7%D7%A8%D7%99%D7%A4%D7%98%D7%95%D7%92%D7%A8%D7%A4%D7%99%D7%94
adding to data set URL: https://he.wikipedia.org/wiki/%D7%9E%D7%92%D7%93%D7%9C%D7%99_%D7%94%D7%90%D7%A0%D7%95%D7%99
adding to data set URL: https://he.wikipedia.org/wiki/%D7%90%D7%93%D7%95%D7%95%D7%99%D7%9F_%D7%94%D7%90%D7%91%D7%9C
adding to data set URL: https://he.wikipedia.org/wiki/%D7%94%D7%A9%D7%93%D7%94_%D7%94%D7%9E%D7%92%D7%A0%D7%98%D7%99_%D7%A9%D7%9C_%D7%9B%D7%93%D7%95%D7%A8_%D7%94%D7%90%D7%A8%D7%A5
adding to data set URL:

adding to data set URL: https://he.wikipedia.org/wiki/%D7%97%D7%95%D7%9E%D7%95%D7%AA_%D7%99%D7%A8%D7%95%D7%A9%D7%9C%D7%99%D7%9D
adding to data set URL: https://he.wikipedia.org/wiki/%D7%9E%D7%A0%D7%94%D7%A8%D7%95%D7%AA_%D7%94%D7%9B%D7%95%D7%AA%D7%9C
adding to data set URL: https://he.wikipedia.org/wiki/%D7%90%D7%A1%D7%A4%D7%A7%D7%AA_%D7%94%D7%9E%D7%99%D7%9D_%D7%9C%D7%99%D7%A8%D7%95%D7%A9%D7%9C%D7%99%D7%9D
adding to data set URL: https://he.wikipedia.org/wiki/%D7%9B%D7%A0%D7%A1%D7%99%D7%99%D7%AA_%D7%94%D7%A7%D7%91%D7%A8
adding to data set URL: https://he.wikipedia.org/wiki/%D7%99%D7%A8%D7%95%D7%A9%D7%9C%D7%99%D7%9D_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%94_%D7%94%D7%9E%D7%9E%D7%9C%D7%95%D7%9B%D7%99%D7%AA
adding to data set URL: https://he.wikipedia.org/wiki/%D7%99%D7%A8%D7%95%D7%A9%D7%9C%D7%99%D7%9D_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%94_%D7%94%D7%A6%D7%9C%D7%91%D7%A0%D7%99%D7%AA
adding to data set URL: https://he.wikipedia.org/wiki/%D7%99%D7%A8%D7%95%D7%A9%D7%9C%D7%99%D7%9D_%D7%91%D7%AA%D7%A7

ValueError: substring not found

## POS tagging the text

The POS of a word considered to be a useful feature for labeling. As we don't have the true POS of each word we create the POS ourselfs. We do this by Using The Hebrew POS tagger Of Meni as described [here](https://www.cs.bgu.ac.il/~elhadad/nlpproj/LDAforHebrew.html)

Run this block **only if you downloaded and installed Meni's Hebrew Tagger**

cleaning data - After running this cell, tags will be an array of pairs [word, tag] for every word in corpus

## Third part - Moshe1 take it from here

In [4]:
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer


import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(x_train, y_train)



CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=0.1, c2=0.1,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

In [11]:
urlList = ["https://he.wikipedia.org/wiki/%D7%A4%D7%99%D7%9C%D7%95%D7%A1%D7%95%D7%A4%D7%99%D7%94_%D7%A9%D7%9C_%D7%94%D7%9E%D7%93%D7%A2"]
data = generate_dataset(urlList,"test_data")

x_test = [x[0] for x in data]
y_test = [x[1] for x in data]
y_pred = crf.predict(x_test)
print("test set size: " + str(len(x_test)))
print("---------------------------------------------------")
for i in range(len(y_test[0])):
    print(str(i) + " -  truth: " + y_test[0][i] + " prediction: " + y_pred[0][i])
print("---------------------------------------------------")
for i in range(len(y_test[1])):
    print(str(i) + " -  truth: " + y_test[1][i] + " prediction: " + y_pred[1][i])
print("---------------------------------------------------")
for i in range(len(y_test[2])):
    print(str(i) + " -  truth: " + y_test[2][i] + " prediction: " + y_pred[2][i])

adding to data set URL: https://he.wikipedia.org/wiki/%D7%A4%D7%99%D7%9C%D7%95%D7%A1%D7%95%D7%A4%D7%99%D7%94_%D7%A9%D7%9C_%D7%94%D7%9E%D7%93%D7%A2
test set size: 13
---------------------------------------------------
0 -  truth: none prediction: none
1 -  truth: none prediction: none
2 -  truth: none prediction: none
3 -  truth: none prediction: none
4 -  truth: none prediction: none
5 -  truth: none prediction: none
6 -  truth: none prediction: none
7 -  truth: , prediction: none
8 -  truth: none prediction: none
9 -  truth: none prediction: none
10 -  truth: none prediction: none
11 -  truth: none prediction: none
12 -  truth: . prediction: none
13 -  truth: none prediction: none
14 -  truth: none prediction: none
15 -  truth: none prediction: none
16 -  truth: none prediction: none
17 -  truth: none prediction: none
18 -  truth: , prediction: none
19 -  truth: none prediction: none
20 -  truth: , prediction: none
21 -  truth: none prediction: none
22 -  truth: none prediction: none


In [12]:
labels = list(crf.classes_)
labels.remove('none')
y_pred = crf.predict(x_test)
metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)

c:\users\moshe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\moshe\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.20533385339216498